# 日期数据

## 格式转换


In [4]:
import pandas as pd

df_kindsof_date = pd.DataFrame({
    '日期': ['07-Oct-24', '10/07/2024', '2024.10.07', '2024/10/07', '20241007']
})

df_kindsof_date['新日期'] = pd.to_datetime(df_kindsof_date['日期'])
df_kindsof_date

C:\Users\lianb\AppData\Local\Temp\ipykernel_20596\3492425423.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_kindsof_date['新日期'] = pd.to_datetime(df_kindsof_date['日期'])


,日期,新日期
0,07-Oct-24,2024-10-07
1,10/07/2024,2024-10-07
2,2024.10.07,2024-10-07
3,2024/10/07,2024-10-07
4,20241007,2024-10-07


In [5]:
print(type(df_kindsof_date.loc[1, '日期']))
print(type(df_kindsof_date.loc[1, '新日期']))

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


# 时间数据

## 重采样

- https://pandas.pydata.org/docs/reference/api/pandas.Series.resample.html#pandas.Series.resample
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html#pandas.DataFrame.resample

### 降采样

In [8]:
import pandas as pd

s = pd.Series(
    range(9),
    index = pd.date_range('2024-10-07', periods=9, freq='min')
)
s

2024-10-07 00:00:00    0
2024-10-07 00:01:00    1
2024-10-07 00:02:00    2
2024-10-07 00:03:00    3
2024-10-07 00:04:00    4
2024-10-07 00:05:00    5
2024-10-07 00:06:00    6
2024-10-07 00:07:00    7
2024-10-07 00:08:00    8
Freq: min, dtype: int64

In [17]:
s.resample('3min').sum()

2024-10-07 00:00:00     3
2024-10-07 00:03:00    12
2024-10-07 00:06:00    21
Freq: 3min, dtype: int64

### 升采样

In [18]:
import pandas as pd

s = pd.Series(
    range(2),
    index = pd.date_range('2024-10-07', periods=2)
)
s

2024-10-07    0
2024-10-08    1
Freq: D, dtype: int64

In [22]:
# 默认不填充
s.resample('6h').asfreq()

2024-10-07 00:00:00    0.0
2024-10-07 06:00:00    NaN
2024-10-07 12:00:00    NaN
2024-10-07 18:00:00    NaN
2024-10-08 00:00:00    1.0
Freq: 6h, dtype: float64

In [25]:
# 前向填充：forhand
s.resample('6h').ffill()

2024-10-07 00:00:00    0
2024-10-07 06:00:00    0
2024-10-07 12:00:00    0
2024-10-07 18:00:00    0
2024-10-08 00:00:00    1
Freq: 6h, dtype: int64

In [26]:
# 后向填充：backhand
s.resample('6h').bfill()

2024-10-07 00:00:00    0
2024-10-07 06:00:00    1
2024-10-07 12:00:00    1
2024-10-07 18:00:00    1
2024-10-08 00:00:00    1
Freq: 6h, dtype: int64

## `ohlc()`

In [31]:
import pandas as pd

s = pd.Series(
    range(12),
    index = pd.date_range('2024-10-07', periods=12, freq='min')
)
s

2024-10-07 00:00:00     0
2024-10-07 00:01:00     1
2024-10-07 00:02:00     2
2024-10-07 00:03:00     3
2024-10-07 00:04:00     4
2024-10-07 00:05:00     5
2024-10-07 00:06:00     6
2024-10-07 00:07:00     7
2024-10-07 00:08:00     8
2024-10-07 00:09:00     9
2024-10-07 00:10:00    10
2024-10-07 00:11:00    11
Freq: min, dtype: int64

In [32]:
s.resample('5min').ohlc()

,open,high,low,close
2024-10-07 00:00:00,0,4,0,4
2024-10-07 00:05:00,5,9,5,9
2024-10-07 00:10:00,10,11,10,11


## 移动窗口

In [43]:
import pandas as pd
import numpy as np

df_window = pd.DataFrame(
    {'value': np.random.randint(0, 100, 9)},
    index = pd.date_range('2024-10-07', periods=9)
)
df_window

,value
2024-10-07,69
2024-10-08,19
2024-10-09,20
2024-10-10,90
2024-10-11,76
2024-10-12,1
2024-10-13,50
2024-10-14,71
2024-10-15,9


In [46]:
# 没有使用`mmin_periods`时，当数量小于观察值数量时为NaN
df_window['mean'] = df_window['value'].rolling(3).mean()
df_window

,value,mean
2024-10-07,69,NaN
2024-10-08,19,NaN
2024-10-09,20,36.000000
2024-10-10,90,43.000000
2024-10-11,76,62.000000
2024-10-12,1,55.666667
2024-10-13,50,42.333333
2024-10-14,71,40.666667
2024-10-15,9,43.333333


In [47]:
# 没有使用`mmin_periods`时，当数量小于观察值数量时为NaN
df_window['mean'] = df_window['value'].rolling(3, min_periods=1).mean()
df_window

,value,mean
2024-10-07,69,69.000000
2024-10-08,19,44.000000
2024-10-09,20,36.000000
2024-10-10,90,43.000000
2024-10-11,76,62.000000
2024-10-12,1,55.666667
2024-10-13,50,42.333333
2024-10-14,71,40.666667
2024-10-15,9,43.333333
